In [1]:
import whisper
import openai
import json
import pyaudio
import wave
import os
import functools
import subprocess

/home/logan_18/anaconda3/envs/peppercudaenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# openai.api_key=
# conversation=[{"role":"system","content":"You are a helpful assistant"}]

# while(1):
#     user_input=input("Enter your question: ")
#     conversation.append({"role":"user","content": user_input})
#     response=openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",
#         messages=conversation,
#         temperature=0.2,
#         max_tokens=1000,
#         top_p=0.9
#     )
#     conversation.append({"role":"assistant","content":response['choices'][0]['message']['content']})
#     answer=response['choices'][0]['message']['content']
#     # print("\n" + answer + "\n")
#     confirmation=input("Do you wish to continue asking questions? Say yes or no please.")
#     if(confirmation=="y" or confirmation=="Y"):
#         continue
#     elif(confirmation=="n" or confirmation=="N"):
#         break




In [4]:
print(conversation)

[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'who is the president of the united states?'}, {'role': 'assistant', 'content': 'As an AI language model, I cannot provide real-time information. However, as of my last training data, the current President of the United States is Joe Biden.'}, {'role': 'user', 'content': 'who is his wife'}, {'role': 'assistant', 'content': 'The wife of President Joe Biden is Jill Biden.'}]


In [11]:
# user_input=input("Enter the question: ")
# openai.api_key=
# conversation=[{"role":"system","content":"You are a helpful assistant"}]
# completion=openai.Completion.create(engine="text-davinci-003",prompt=user_input,max_tokens=1000)
# response=completion.choices[0]['text']
# print(response)

In [12]:
# Function to load the whisper model
@functools.lru_cache(maxsize=None)
def load_whisper_model():
    whisper_model=whisper.load_model("large")
    return whisper_model

# Function to transcribe text from audio
def transcribe(audio_file):
    whisper_model=load_whisper_model()
    transcription = whisper_model.transcribe(audio_file)
    return transcription['text']

# Function to record audio
def record_audio(path, filename, duration):
    # Set the parameters for the audio stream
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    fs = 44100
    
    # Initialize the PyAudio object
    p = pyaudio.PyAudio()
    
    # Open the audio stream
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)
    
    frames = []
    
    # Record the audio for the specified duration
    for i in range(int(fs / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    
    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    
    # Terminate the PyAudio object
    p.terminate()
    
    # Save the recorded audio as a WAV file
    file_path = os.path.join(path, filename)
    wf = wave.open(file_path, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [13]:
def chatgpt_response(question):
    load_whisper_model()
    # using the openai api key
    openai.api_key=
    conversation=[{"role":"system","content":"You are a helpful assistant"}]
    # completion=openai.Completion.create(engine="text-davinci-003",prompt=question,max_tokens=1000)
    # response=completion.choices[0]['text']

    while(1):
        conversation.append({"role":"user","content": question})
        response=openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=conversation,
            temperature=0.2,
            max_tokens=1000,
            top_p=0.9
        )
        conversation.append({"role":"assistant","content":response['choices'][0]['message']['content']})
        answer=response['choices'][0]['message']['content']
        # print("\n" + response['choices'][0]['message']['content'] + "\n")
        writing_response_to_json_file(answer)
        #subprocess.run(['bash','chatgpt.sh'])
        confirmation=input("Do you wish to continue asking questions? Say yes or no please.")
        if(confirmation=="yes" or confirmation=="YES"):
            print("What do you want to know? ")
            record_audio("/home/logan_18/pepper/chatgpt/recordings", "recording.mp3", 7)
            print("Question recorded")
            print("Transcribing audio")
            question=transcribe("/home/logan_18/pepper/chatgpt/recordings/recording.mp3")
            print("Audio Transcribed and question generated")
            print(question)
            # chatgpt_response(transcribed_text)
        elif(confirmation=="no" or confirmation=="NO" or confirmation=="nO" or confirmation=="No"):
            break
        else:
            print("Please enter either yes or no!!!!")
    



def writing_response_to_json_file(answer):
    sorted_output=json.dumps(answer)
    with open('/home/logan_18/pepper/chatgpt/json_file.json', "w") as outfile:
        outfile.write(sorted_output)

        
    
    

In [16]:
import zmq

: 

In [14]:
def main():
    # load_whisper_model()
    # while(1):
    print("What do you want to know?")
    # Record 7 seconds of audio
    record_audio("/home/logan_18/pepper/chatgpt/recordings", "recording.mp3", 7)
    print("Question recorded!!")
    # Transcribing audio
    print("Transcribing audio")
    result=transcribe("/home/logan_18/pepper/chatgpt/recordings/recording.mp3")
    print("Audio Transcribed and Question Generated: " + result)
    chatgpt_response(result)
    # Running the shell script to execute the code on pepper
    # subprocess.run(['bash','chatgpt.sh'])
    # execute=input("Do you want to ask another question? If Yes: Press Y or y, If No: Press N or n---> ")
    # if(execute=='Y' or execute=='y'):
    #     continue
    # elif(execute=='N' or execute=='n'):
    #     return
    # else:
    #     print("Please enter valid option")



if __name__ == "__main__":
    main()

What do you want to know?


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5007:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error

Question recorded!!
Question generated:  Tell me something about California.
What do you want to know? 


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5007:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error

Question recorded
Transcribing audio
Audio Transcribed and question generated
